<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6. Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

/opt/files/Workspace/virtualenv/python3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/files/Workspace/virtualenv/python3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=7) и отмасштабируйте данные с помощью StandardScaler.**

In [5]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [6]:
linreg = LinearRegression(n_jobs=-1)
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

**Оцените среднеквадратичную ошибку модели на обучающей и отложенной выборках.**

In [7]:
print('MSE на обучающей выборке: {}'.format(mean_squared_error(y_train, linreg.predict(X_train_scaled))))
print('MSE на отложенной выборке: {}'.format(mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled))))

MSE на обучающей выборке: 0.5661467734569134
MSE на отложенной выборке: 0.5568755310945736


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**

In [8]:
linreg_coef = pd.DataFrame(np.abs(linreg.coef_), X.columns, columns=['Weight'])
linreg_coef.sort_values(by='Weight', ascending=False)

,Weight
density,0.423776
residual sugar,0.402290
alcohol,0.252922
volatile acidity,0.185992
pH,0.095215
sulphates,0.071056
free sulfur dioxide,0.058823
fixed acidity,0.049315
total sulfur dioxide,0.012501
chlorides,0.002624


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=7.**

In [9]:
lasso1 = Lasso(alpha=0.01, random_state=7)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=7,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [10]:
lasso1_coef = pd.DataFrame(np.abs(lasso1.coef_), X.columns, columns=['Weight'])
lasso1_coef.sort_values(by='Weight', ascending=False)

,Weight
alcohol,0.364161
residual sugar,0.209587
volatile acidity,0.186631
density,0.155455
free sulfur dioxide,0.047046
sulphates,0.045213
pH,0.037394
fixed acidity,0.006228
chlorides,0.003843
total sulfur dioxide,0.000083


"Отвалилcя" признак __citric acid__.

**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=7.**

In [11]:
alphas = np.linspace(0.1, 10, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=7, n_jobs=-1)
 
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([  0.1    ,   0.14975, ...,   9.95025,  10.     ]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=7, selection='cyclic', tol=0.0001,
    verbose=False)

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. Какие 2 признака оказались главными?**

In [12]:
lasso_cv_coef = pd.DataFrame(np.abs(lasso_cv.coef_), X.columns, columns=['Weight'])
lasso_cv_coef.sort_values(by='Weight', ascending=False)

,Weight
alcohol,0.286830
volatile acidity,0.091039
fixed acidity,0.000000
citric acid,0.000000
residual sugar,0.000000
chlorides,0.000000
free sulfur dioxide,0.000000
total sulfur dioxide,0.000000
density,0.000000
pH,0.000000


Главными оказались признаки __alcohol__ и __volatile acidity__.

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**

In [13]:
print('MSE на обучающей выборке: {}'.format(mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))
print('MSE на оставленной выборке: {}'.format(mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))))

MSE на обучающей выборке: 0.61911775135283
MSE на оставленной выборке: 0.6186331276479452


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=7.**

In [14]:
forest = RandomForestRegressor(random_state=7)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

**Оцените среднеквадратичную ошибку модели на обучающей выборке, на кросс-валидации (cross_val_score с scoring='mean_squared_error' и остальными параметрами по умолчанию) и на оставленной выборке.**

In [15]:
print('MSE на обучающей выборке: {}'.format(mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))
print('Среднее MSE на кросс-валидации: {}'\
      .format(np.abs(np.mean(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))))
print('MSE на оставленной выборке: {}'.format(mean_squared_error(y_holdout, forest.predict(X_holdout_scaled))))

MSE на обучающей выборке: 0.61911775135283
Среднее MSE на кросс-валидации: 0.47708556461090357
MSE на оставленной выборке: 0.4107142857142857


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на оставленной выборке.**

In [16]:
%%time
forest_params = {'max_depth': list(range(15, 20)), 
                 'min_samples_leaf': list(range(2, 8))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=7), 
                                 forest_params, 
                                 scoring='neg_mean_squared_error',  
                                 n_jobs=-1, cv=5)
locally_best_forest.fit(X_train_scaled, y_train)

print('Среднее MSE на кросс-валидации: {}'\
      .format(np.abs(np.mean(cross_val_score(
                    locally_best_forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))))
print('MSE на оставленной выборке: {}'\
      .format(mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled))))

Среднее MSE на кросс-валидации: 0.4774404523602887
MSE на оставленной выборке: 0.4104282432482156
CPU times: user 2.4 s, sys: 272 ms, total: 2.67 s
Wall time: 19.1 s


**Оцените важность признаков с помощью случайного леса.**

In [17]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, X.columns, columns=['Weight'])
rf_importance.sort_values(by='Weight', ascending=False)

,Weight
alcohol,0.251904
volatile acidity,0.128601
free sulfur dioxide,0.117733
pH,0.075218
total sulfur dioxide,0.072407
residual sugar,0.064290
density,0.062162
sulphates,0.060817
chlorides,0.057891
citric acid,0.055702


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

Самыми влиятельными признаками оказались __alcohol__ и __volatile acidity__, самым невлиятельным - __citric acid__. Это подтверждает и Lasso и Random Forest.

Лучшей моделью (на оставленной выборке) оказалась модель __Random Forest с настроенными на кросс-валидации параметрами__.